In [11]:
#import nessary libraries

from bs4 import BeautifulSoup as bs
import requests
import smtplib
import time
import datetime


In [12]:
#connecting to the website and scrape all the data from the website and take into local

url='https://www.amazon.com/Data-Analyst-T-Shirt-Male-Boss/dp/B09T5JC2XM/ref=sr_1_1?crid=3QE0W4BK4GHFC&keywords=data%2Banalyst%2Bshirt&qid=1683299953&sprefix=data%2Banalyst%2Bs%2Caps%2C689&sr=8-1&th=1&psc=1'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page=requests.get(url,headers=headers)
soup1=bs(page.content,"html.parser")   #pulling all the html from the website
soup2=bs(soup1.prettify(),"html.parser")   #reading html data with better indendations

title=soup2.find(id='productTitle').get_text()
price=soup2.find(id="corePriceDisplay_desktop_feature_div").get_text()
# print(price)
# print(title)




KeyboardInterrupt: 

In [ ]:
#cut and strip with slicing required data from the web
price=price.strip()[1:6]  
title=title.strip()

In [ ]:

today=datetime.date.today()

In [ ]:
#creating csv files and storing data in the csv file
import csv

headers=['Title','Price','Date']
data=[title,price,today]

with open('Amazon_Web_Scraper_Dataset.csv','w',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(headers)
    writer.writerow(data)



In [25]:
#Creating dataframes to store data in the csv

import pandas as pd

df=pd.read_csv(r'C:\Users\tuanminhaj\Data-Analyst-Projects\WebScraping-Project\Amazon_Web_Scraper_Dataset.csv')
df


,Title,Price,Date
0,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
1,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
2,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05


In [24]:
#Appending data to the csv 

with open('Amazon_Web_Scraper_Dataset.csv','a+',newline='',encoding='UTF8') as f:
    writer=csv.writer(f)
    writer.writerow(data)

In [29]:
#Automating appending data to insert into dataframe itself within given time boundary

def check_price():
  
    url='https://www.amazon.com/Data-Analyst-T-Shirt-Male-Boss/dp/B09T5JC2XM/ref=sr_1_1?crid=3QE0W4BK4GHFC&keywords=data%2Banalyst%2Bshirt&qid=1683299953&sprefix=data%2Banalyst%2Bs%2Caps%2C689&sr=8-1&th=1&psc=1'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    page=requests.get(url,headers=headers)
    soup1=bs(page.content,"html.parser")   
    soup2=bs(soup1.prettify(),"html.parser")   

    title=soup2.find(id='productTitle').get_text()
    price=soup2.find(id="corePriceDisplay_desktop_feature_div").get_text()

    price=price.strip()[1:6]  
    title=title.strip()

    today=datetime.date.today()
    
    headers=['Title','Price','Date']
    data=[title,price,today]

    with open('Amazon_Web_Scraper_Dataset.csv','a+',newline='',encoding='UTF8') as f:
        writer=csv.writer(f)
        writer.writerow(data)

    

In [30]:
while True:
    check_price()
    time.sleep(5)

KeyboardInterrupt: 

In [31]:
df=pd.read_csv(r'C:\Users\tuanminhaj\Data-Analyst-Projects\WebScraping-Project\Amazon_Web_Scraper_Dataset.csv')
df

,Title,Price,Date
0,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
1,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
2,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
3,Title,Price,Date
4,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
5,Title,Price,Date
6,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
7,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
8,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
9,"Data Analyst T-Shirt for Men,Him, Male, Boss, ...",25.75,2023-05-05
